# Nuevos Planes Megaline 

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

## Introducción

En este proyecto, desarrollaremos un modelo de clasificación para recomendar uno de los nuevos planes de Megaline: Smart o Ultra. Utilizaremos datos de comportamiento de los clientes que ya han cambiado a los nuevos planes.

**Descripción de datos**

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

* __сalls__ — número de llamadas,
* __minutes__ — duración total de la llamada en minutos,
* __messages__ — número de mensajes de texto,
* __mb_used__ — Tráfico de Internet utilizado en MB,
* __is_ultra__ — plan para el mes actual (Ultra - 1, Smart - 0).

## Cargar y Examinar el Conjunto de Datos
Cargaremos el conjunto de datos y revisaremos sus primeros registros para entender su estructura y tipos de datos.

In [1]:
# Importar librerias
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Cargar el conjunto de datos
df = pd.read_csv('users_behavior.csv')

# Mostrar las primeras filas del conjunto de datos
df.head()

Los datos se han cargado correctamente. Cada fila representa el comportamiento de un usuario en términos de llamadas, minutos, mensajes y uso de datos (MB). La variable __is_ultra__ indica si el usuario está en el plan __Ultra (1__) o __Smart (0)__.

## Revisión y Corrección de Tipos de Datos
Revisaremos los tipos de datos de cada columna y corregiremos los tipos de las columnas `calls` y `messages` a enteros.

In [3]:
# Revisar los tipos de datos de cada columna
df.dtypes

In [4]:
# Corregir los tipos de datos de `calls` y `messages` a enteros

df['calls'] = df['calls'].astype(int)
df['messages'] = df['messages'].astype(int)

# Verificar la corrección
df.dtypes

Los tipos de datos de las columnas __calls__ y __messages__ se han corregido de __float64__ a __int32__, asegurando que los datos sean del tipo correcto para el análisis posterior.

## División del Conjunto de Datos
Dividimos el conjunto de datos en conjuntos de entrenamiento (60%), validación (20%) y prueba (20%).

In [5]:
# Separar características y la variable objetivo
X = df.drop(columns=['is_ultra'])
y = df['is_ultra']

In [6]:
# Dividir los datos: 60% entrenamiento, 20% prueba, 20% validación
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Mostrar los tamaños de las divisiones
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de validación: {X_val.shape}")
print(f"Tamaño del conjunto de prueba: {X_test.shape}")

Los datos se han dividido en conjuntos de entrenamiento, validación y prueba, con **1928** observaciones para **entrenamiento** , **643** observaciones para **validación** y **643** para **prueba**.Esto asegura que podemos entrenar los modelos en el conjunto de entrenamiento, ajustar hiperparámetros en el conjunto de validación, y evaluar el rendimiento final en el conjunto de prueba.

## Búsqueda de Hiperparámetros
Realizamos una búsqueda en cuadrícula para encontrar los mejores hiperparámetros para los modelos de clasificación binaria DecisionTreeRegressor, RandomForestRegressor y LogisticRegression.

In [7]:
from sklearn.model_selection import RandomizedSearchCV

# Definir los modelos y sus hiperparámetros para la búsqueda en cuadrícula
modelos = {
    'Decision Tree': {
        'modelo': DecisionTreeClassifier(),
        'params': {
            'max_depth': [10, 20, 30]
        }
    },
    'Random Forest': {
        'modelo': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [10, 20]
        }
    },
    'Logistic Regression': {
        'modelo': LogisticRegression(solver='liblinear'),
        'params': {
            'C': [0.01, 0.1, 1, 10, 100]
        }
    }
}

# Realizar búsqueda en cuadrícula para encontrar los mejores hiperparámetros para cada modelo utilizando entrenamiento y validación
mejores_modelos = {}
for nombre_modelo, info_modelo in modelos.items():
    grid_search = GridSearchCV(info_modelo['modelo'], info_modelo['params'], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    mejores_modelos[nombre_modelo] = grid_search.best_estimator_
    val_score = grid_search.score(X_val, y_val)
    print(f"{nombre_modelo} mejores params: {grid_search.best_params_}")
    print(f"{nombre_modelo} exactitud de validación: {val_score:.4f}")

mejores_modelos

__Conclusiones__:

* __Decision Tree__: El mejor parámetro es __max_depth: 10__ con un accuracy de __0.7932__.
* __Random Forest__: Los mejores parámetros son __n_estimators: 100__ y __max_depth: 10__ con accuracy de __0.8072__.
* __Logistic Regression__: Los mejores parámetros son __C=1__ y __solver: liblinear__ con un accuracy de __0.7201__.

## Evaluación de los Modelos
Evaluamos los mejores modelos en el conjunto de prueba para medir su rendimiento.

In [8]:
# Mejores modelos
mejor_dt = mejores_modelos['Decision Tree']
mejor_rf = mejores_modelos['Random Forest']
mejor_lr = mejores_modelos['Logistic Regression']

# Entrenar los modelos
mejor_dt.fit(X_train, y_train)
mejor_rf.fit(X_train, y_train)
mejor_lr.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_dt = mejor_dt.predict(X_test)
y_pred_rf = mejor_rf.predict(X_test)
y_pred_lr = mejor_lr.predict(X_test)

# Calcular la precisión
accuracy_dt = accuracy_score(y_test, y_pred_dt)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
accuracy_lr = accuracy_score(y_test, y_pred_lr)

print(f"Precisión de Decision Tree: {accuracy_dt:.4f}")
print(f"Precisión de Random Forest: {accuracy_rf:.4f}")
print(f"Precisión de Logistic Regression: {accuracy_lr:.4f}")

In [9]:
# Seleccionar el mejor modelo basado en la exactitud de validación
mejor_modelo_nombre = max(mejores_modelos, key=lambda nombre: mejores_modelos[nombre].score(X_val, y_val))
mejor_modelo = mejores_modelos[mejor_modelo_nombre]
print(f"Mejor modelo: {mejor_modelo_nombre}")

El modelo **Random Forest** con accuracy de **0.8040** es el más preciso en el conjunto de prueba, seguido por el **Decision Tree** y luego la **Logistic Regression**.

In [10]:
# Entrenar el mejor modelo en el conjunto de entrenamiento completo
mejor_modelo.fit(X_train, y_train)

# Evaluar el mejor modelo en el conjunto de validación
val_pred = mejor_modelo.predict(X_val)
val_accuracy = accuracy_score(y_val, val_pred)
print(f"Precisión del mejor modelo en el conjunto de validación: {val_accuracy:.4f}")

# Evaluar el mejor modelo en el conjunto de prueba
test_pred = mejor_modelo.predict(X_test)
test_accuracy = accuracy_score(y_test, test_pred)
print(f"Precisión del mejor modelo en el conjunto de prueba: {test_accuracy:.4f}")

Observamos que el modelo __Random Forest__ se ajusta bastante bien tanto para el conjunto de validación como para el conjunto de prueba, obteniendose un accuracy de __0.8072__ y __0.8258__.

## Prueba de Cordura
Realizamos una prueba de cordura utilizando un clasificador **dummy** que **predice** la **clase más frecuente** y comparamos su rendimiento con el modelo **Random Forest**.

In [11]:
# Prueba de cordura con un clasificador dummy
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(X_train, y_train)
y_pred_dummy = dummy_clf.predict(X_test)
accuracy_dummy = accuracy_score(y_test, y_pred_dummy)

print(f"Precisión del Clasificador Dummy: {accuracy_dummy:.4f}")

El clasificador dummy obtuvo una exactitud de **70%**.

### Conclusiones Generales:
- Se desarrolló un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud era 0.75, y el modelo `Random Forest` superó este umbral con una precisión del __`82%`__.

- La segmentación de los datos en conjuntos de entrenamiento, validación y prueba se realizó adecuadamente, utilizando un 60% para entrenamiento, 20% para validación y 20% para prueba.

- La búsqueda de hiperparámetros permitió optimizar los modelos, identificando los mejores parámetros para `Decision Tree`, `Random Forest`, y `Logistic Regression`.

- El modelo `Random Forest` demostró ser el más efectivo en términos de precisión.

- La prueba de cordura con el clasificador dummy confirmó la validez del modelo `Random Forest`.